A confiança nos resultados que você obtém ao analisar dados pode ser influenciada por várias questões relacionadas à qualidade e representatividade das informações. Aqui estão os principais fatores que podem impactar esses resultados:

Dados Ausentes (ou Missing Values):

Se muitos dados estão faltando em uma coluna, como em uma pesquisa onde várias pessoas deixaram algumas perguntas sem resposta, as médias e outros cálculos podem ficar distorcidos.

O que fazer: Preencha essas lacunas com valores médios, medianas ou, em alguns casos, preveja o que poderia estar lá com base em outros dados. Outra opção é remover completamente essas linhas ou colunas se elas estiverem muito incompletas.
Valores Extremamente Diferentes (Outliers):

Outliers são dados que fogem muito do padrão, como uma pessoa que ganha muito mais ou muito menos que a maioria em uma pesquisa de renda. Esses valores podem afetar bastante a média e o desvio padrão, dando uma impressão errada dos dados.

O que fazer: Analise esses valores para entender se são erros ou informações reais. Depois, decida se eles devem ser removidos ou mantidos.
Tamanho da Amostra:

Com poucos dados, as conclusões podem não refletir bem a realidade, pois a média e outras métricas ficam muito sensíveis a pequenas mudanças.

O que fazer: Sempre que possível, colete mais dados. Se isso não for possível, trate os resultados com cuidado, pois eles podem não ser tão precisos.
Distribuição dos Dados:

Quando os dados estão "desequilibrados", como se a maioria dos valores estivesse mais concentrada de um lado, a média pode não representar bem a situação.

O que fazer: Considere usar a mediana, que pode dar uma visão mais clara quando os dados estão assimétricos.
Erros de Medição:

Erros durante a coleta ou registro dos dados, como valores digitados incorretamente, podem afetar os resultados e gerar conclusões erradas.

O que fazer: Revise os dados e corrija ou elimine os erros.
Amostra Tendenciosa (ou Falta de Representatividade):

Se a amostra coletada não representa bem o grupo que você quer estudar (como coletar dados de uma cidade e querer aplicar as conclusões para um país inteiro), os resultados podem ser enganosos.

O que fazer: Certifique-se de que a amostra inclui todos os grupos importantes do estudo. Se não for possível, tome cuidado ao interpretar os resultados.
Escalas e Unidades Inconsistentes:

Comparar valores em escalas diferentes (como mil reais versus milhões de reais) ou em unidades diferentes (como reais e dólares) pode levar a distorções.

O que fazer: Ajuste os dados para que todos estejam na mesma escala ou unidade antes de compará-los.
Atualidade dos Dados:

Dados muito antigos podem não refletir o cenário atual e levar a conclusões que já não fazem mais sentido.

O que fazer: Sempre que possível, trabalhe com dados atualizados para que a análise reflita a situação mais recente.

### Importando as bibliotecas necessárias

In [1]:
# Importação das bibliotecas
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import locale
    import openpyxl
    import re
except ImportError:
    # Instalação condicional
    import sys
    !{sys.executable} -m pip install pandas numpy matplotlib seaborn locale openpyxl re



#### Carregando o arquivo CSV em um DataFrame
#### Neste passo, carregaremos o arquivo CSV com os dados dos anúncios em um DataFrame do pandas
#### Certifique-se de ter o arquivo 'miro_dados_brutos_desde_janeiro.csv' no mesmo diretório deste notebook

In [2]:
dados_anuncios = pd.read_csv('miro_dados_brutos_desde_janeiro.csv')

In [3]:
# Exibe as primeiras 5 linhas do dataset
print("Primeiras 5 linhas do dataset:")
display(dados_anuncios.head())

Primeiras 5 linhas do dataset:


,Nome do anúncio,Alcance,Impressões,Frequência,Valor usado (BRL),Configuração de atribuição,Tipo de resultado,Resultados,Custo por resultado,CPM (custo por 1.000 impressões),...,CPC (custo por clique no link),Cliques (todos),Engajamento com a Página,Reproduções de 25% do vídeo,Reproduções de 50% do vídeo,Reproduções de 75% do vídeo,Reproduções de 95% do vídeo,Reproduções de 100% do vídeo,Início dos relatórios,Término dos relatórios
0,AD 01 - CARROSSEL FEED - 05/06/2024,167158,409103,2.447403,562.85,Clique de 7 dias ou visualização de 1 dia,Alcance,167158.0,3.367174,1.375815,...,1.710790,625,381.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
1,AD 01 - CARROSSEL - 18/06/2024,120917,253308,2.094892,640.82,Clique de 7 dias ou visualização de 1 dia,Alcance,120917.0,5.299668,2.529806,...,2.926119,481,262.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
2,AD 01 - VIDEO - 16/09/2024,95498,178060,1.864542,340.81,Clique de 7 dias ou visualização de 1 dia,Alcance,95498.0,3.568766,1.914018,...,170.405000,159,6643.0,4553.0,1840.0,1163.0,890.0,839.0,2024-01-01,2024-10-30
3,AD 01 - VIDEO - 01/10/2024,81964,159631,1.947575,297.95,Clique de 7 dias ou visualização de 1 dia,Alcance,81964.0,3.635133,1.866492,...,148.975000,124,4955.0,2660.0,1178.0,742.0,206.0,189.0,2024-01-01,2024-10-30
4,AD 01 - STORY - 16/10/2024,59677,74129,1.242170,119.48,Clique de 7 dias ou visualização de 1 dia,Alcance,59677.0,2.002111,1.611785,...,NaN,35,5.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30


In [4]:
# Exibe tipos de dados de cada coluna
print("\nResumo das colunas e tipos de dados:")
display(dados_anuncios.dtypes)


Resumo das colunas e tipos de dados:


Nome do anúncio                              object
Alcance                                       int64
Impressões                                    int64
Frequência                                  float64
Valor usado (BRL)                           float64
Configuração de atribuição                   object
Tipo de resultado                            object
Resultados                                  float64
Custo por resultado                         float64
CPM (custo por 1.000 impressões)            float64
CTR (todos)                                 float64
Classificação de qualidade                   object
Classificação da taxa de engajamento         object
Custo por lead                              float64
Valor médio de conversão de compra          float64
Conversas por mensagem iniciadas            float64
Custo por conversa por mensagem iniciada    float64
CPC (custo por clique no link)              float64
Cliques (todos)                               int64
Engajamento 

In [5]:
# Conversão de tipos de dados e verificação de valores ausentes
dados_anuncios = dados_anuncios.convert_dtypes()  # Converte colunas para tipos apropriados automaticamente
print("\nTipos de dados após conversão:")
print(dados_anuncios.dtypes)


Tipos de dados após conversão:
Nome do anúncio                             string[python]
Alcance                                              Int64
Impressões                                           Int64
Frequência                                         Float64
Valor usado (BRL)                                  Float64
Configuração de atribuição                  string[python]
Tipo de resultado                           string[python]
Resultados                                           Int64
Custo por resultado                                Float64
CPM (custo por 1.000 impressões)                   Float64
CTR (todos)                                        Float64
Classificação de qualidade                  string[python]
Classificação da taxa de engajamento        string[python]
Custo por lead                                       Int64
Valor médio de conversão de compra                   Int64
Conversas por mensagem iniciadas                     Int64
Custo por conversa por m

In [6]:
print("\nValores ausentes por coluna:")
print(dados_anuncios.isnull().sum())


Valores ausentes por coluna:
Nome do anúncio                              0
Alcance                                      0
Impressões                                   0
Frequência                                   0
Valor usado (BRL)                            0
Configuração de atribuição                  43
Tipo de resultado                           43
Resultados                                  51
Custo por resultado                         51
CPM (custo por 1.000 impressões)            43
CTR (todos)                                 43
Classificação de qualidade                   0
Classificação da taxa de engajamento         0
Custo por lead                              75
Valor médio de conversão de compra          75
Conversas por mensagem iniciadas            63
Custo por conversa por mensagem iniciada    63
CPC (custo por clique no link)              49
Cliques (todos)                              0
Engajamento com a Página                    45
Reproduções de 25% do vídeo   

### Removendo linhas duplicadas
### Vamos remover duplicatas, pois elas podem distorcer as métricas e análises dos dados de anúncios

In [7]:
# Identifica e exibe as linhas duplicadas antes de removê-las
duplicatas = dados_anuncios[dados_anuncios.duplicated()]

# Verifica se existem duplicatas e exibe uma mensagem
if not duplicatas.empty:
    print("Linhas duplicadas encontradas e que serão removidas:")
    duplicatas
else:
    print("Nenhuma linha duplicada encontrada.")

# Remove as linhas duplicadas
dados_anuncios = dados_anuncios.drop_duplicates()


Linhas duplicadas encontradas e que serão removidas:


### Removendo colunas vazias
### Aqui, removeremos colunas que estejam completamente vazias, pois não contribuem para a análise

In [8]:
dados_anuncios = dados_anuncios.dropna(axis=1, how='all')

In [9]:
# Contagem de valores ausentes por coluna
print("\nContagem de valores ausentes por coluna:")
display(dados_anuncios.isnull().sum())


Contagem de valores ausentes por coluna:


Nome do anúncio                              0
Alcance                                      0
Impressões                                   0
Frequência                                   0
Valor usado (BRL)                            0
Configuração de atribuição                  27
Tipo de resultado                           27
Resultados                                  35
Custo por resultado                         35
CPM (custo por 1.000 impressões)            27
CTR (todos)                                 27
Classificação de qualidade                   0
Classificação da taxa de engajamento         0
Conversas por mensagem iniciadas            47
Custo por conversa por mensagem iniciada    47
CPC (custo por clique no link)              33
Cliques (todos)                              0
Engajamento com a Página                    29
Reproduções de 25% do vídeo                 50
Reproduções de 50% do vídeo                 51
Reproduções de 75% do vídeo                 51
Reproduções d

In [10]:
# Função para extrair a data
def extrair_data(nome_anuncio):
    match = re.search(r'\d{2}/\d{2}/\d{4}', nome_anuncio)
    return match.group(0) if match else None

# Aplicando a função na coluna 'Nome do anúncio' e criando a nova coluna 'Data de criação'
dados_anuncios['Data de criação'] = dados_anuncios['Nome do anúncio'].apply(extrair_data)


# Ajuste de tipos das colunas

In [11]:
# Configura o locale para o Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [12]:

# Função para formatar valores em moeda brasileira
def formatar_moeda(valor):
    return locale.currency(valor, grouping=True) if pd.notnull(valor) else valor

# Função para converter datas para o formato padrão e exibi-las no formato brasileiro
def converter_data(data):
    for fmt in ('%d/%m/%Y', '%Y-%m-%d'):
        try:
            data_formatada = pd.to_datetime(data, format=fmt)
            return data_formatada.strftime('%d/%m/%Y')  # Retorna em formato brasileiro
        except ValueError:
            continue
    return pd.NaT  # Retorna NaT se nenhum formato for adequado

# Aplicando as correções de tipos nas colunas conforme solicitado
dados_anuncios = dados_anuncios.astype({
    'Nome do anúncio': 'string',
    'Alcance': 'int64',
    'Impressões': 'int64',
    'Frequência': 'float64',
    'Configuração de atribuição': 'category',
    'Tipo de resultado': 'category',
    'Resultados': 'float64',
    'Classificação de qualidade': 'category',
    'Classificação da taxa de engajamento': 'category',
    'Cliques (todos)': 'int64',
    'Engajamento com a Página': 'float64',
    'Reproduções de 25% do vídeo': 'float64',
    'Reproduções de 50% do vídeo': 'float64',
    'Reproduções de 75% do vídeo': 'float64',
    'Reproduções de 95% do vídeo': 'float64',
    'Reproduções de 100% do vídeo': 'float64'
})

# Aplicando a função de formatação de moeda nas colunas de valores monetários
for coluna_monetaria in ['Valor usado (BRL)', 'Custo por resultado', 'CPM (custo por 1.000 impressões)', 'CPC (custo por clique no link)']:
    dados_anuncios[coluna_monetaria] = dados_anuncios[coluna_monetaria].apply(formatar_moeda)

# Convertendo datas para o formato brasileiro com a função converter_data
dados_anuncios['Início dos relatórios'] = dados_anuncios['Início dos relatórios'].apply(converter_data)
dados_anuncios['Término dos relatórios'] = dados_anuncios['Término dos relatórios'].apply(converter_data)

# Verificando datas nulas após a conversão
print("Datas nulas em 'Início dos relatórios':")
print(dados_anuncios[dados_anuncios['Início dos relatórios'].isnull()])

print("Datas nulas em 'Término dos relatórios':")
print(dados_anuncios[dados_anuncios['Término dos relatórios'].isnull()])

# Convertendo CTR para float64
dados_anuncios['CTR (todos)'] = pd.to_numeric(dados_anuncios['CTR (todos)'], errors='coerce')


Datas nulas em 'Início dos relatórios':
Empty DataFrame
Columns: [Nome do anúncio, Alcance, Impressões, Frequência, Valor usado (BRL), Configuração de atribuição, Tipo de resultado, Resultados, Custo por resultado, CPM (custo por 1.000 impressões), CTR (todos), Classificação de qualidade, Classificação da taxa de engajamento, Conversas por mensagem iniciadas, Custo por conversa por mensagem iniciada, CPC (custo por clique no link), Cliques (todos), Engajamento com a Página, Reproduções de 25% do vídeo, Reproduções de 50% do vídeo, Reproduções de 75% do vídeo, Reproduções de 95% do vídeo, Reproduções de 100% do vídeo, Início dos relatórios, Término dos relatórios, Data de criação]
Index: []

[0 rows x 26 columns]
Datas nulas em 'Término dos relatórios':
Empty DataFrame
Columns: [Nome do anúncio, Alcance, Impressões, Frequência, Valor usado (BRL), Configuração de atribuição, Tipo de resultado, Resultados, Custo por resultado, CPM (custo por 1.000 impressões), CTR (todos), Classificação d

In [13]:
# Identificar as linhas onde o valor de 'Alcance' é igual a zero
linhas_excluir = dados_anuncios[dados_anuncios['Alcance'] == 0]

# Extrair os nomes dos anúncios que serão excluídos
anuncios_excluidos = linhas_excluir['Nome do anúncio'].tolist()

# Imprimir os nomes dos anúncios que serão excluídos de forma mais bonita
print("Anúncios excluídos:")
for i, anuncio in enumerate(anuncios_excluidos, start=1):
    print(f"{i}. {anuncio}")

# Excluir as linhas onde 'Alcance' é igual a zero do DataFrame original
dados_anuncios = dados_anuncios[dados_anuncios['Alcance'] != 0]

# Resetar o índice após a exclusão
dados_anuncios.reset_index(drop=True, inplace=True)


Anúncios excluídos:
1. AD 04 - VÍDEO - 18 | 04
2. AD 01 - STORIES - 21 | 12
3. AD 03 - CARROSSEL - 17 | 08
4. AD 04 - CARROSSEL 4  - 21 | 12
5. AD 01 - CARROSSEL 1 - 18 | 04
6. AD 02 - CARROSSEL 1 - 18 | 04
7. AD 03 - CARROSSEL 3  - 21 | 12
8. AD 01 IMAGEM ÚNICA - 06 | 10
9. AD 03 - CARROSSEL - 18 | 08
10. AD 03 - IMAGEM ÚNICA - 18 | 04
11. AD 01 - CARROSSEL - 17 | 02
12. AD 01 - IMAGEM ÚNICA - 18 | 08
13. AD 03 - VÍDEO - 18 | 04
14. AD 03 CARROSSEL - 06 | 10
15. AD 02 IMAGEM ÚNICA - 06 | 10
16. AD 02 - IMAGEM ÚNICA - 18 | 08
17. AD 02 - IMAGEM ÚNICA - 17 | 08
18. AD 01 - CARROSSEL 1 - 21 | 12
19. AD 02 - IMAGEM ÚNICA - 14 | 07
20. AD 02 - VÍDEO - 18 | 04
21. AD 01 - VÍDEO - 30 | 06
22. AD 01 - VÍDEO - 18 | 04
23. AD 01 - IMAGEM ÚNICA - 17 | 08
24. AD 02 - CARROSSEL 2 - 18 | 04
25. AD 02 - CARROSSEL 2  - 21 | 12
26. AD 01 - IMAGEM ÚNICA - 14 | 07
27. AD 01 - ESPECIAL - 21 | 12


A tabela mostra as reproduções de vídeo em diferentes estágios: 25%, 50%, 75%, 95% e 100%. Isso ajuda a avaliar o engajamento e retenção do público ao longo do vídeo.

- **Valores acima de zero** indicam o número de vezes que o vídeo foi assistido até certo ponto (25%, 50%, etc.), mostrando que o público estava engajado.
- **Valores zero** (substituídos onde havia `NaN`) indicam ausência de visualizações até aquele ponto em alguns anúncios, sugerindo que o vídeo não conseguiu prender o público.

À medida que as reproduções se aproximam de 100%, uma queda é esperada, indicando perda de interesse ao longo do vídeo. Um número significativo de reproduções até 100% indica um vídeo envolvente. Esses dados ajudam a entender onde o público desiste e se ajustes no conteúdo são necessários.


### Quais são os riscos de substituir valores ausentes (NaN) por zero em uma análise de dados e quais alternativas podem minimizar o viés?

Substituir `NaN` por zero pode enviesar a análise se `NaN` não significar realmente "nenhuma visualização", pois pode indicar dados não coletados. Isso pode reduzir as médias e aumentar a variabilidade, sugerindo que o público desistiu mais cedo do que realmente ocorreu.

### Alternativas:
- **Analisar Separadamente** as campanhas com `NaN`.
- **Imputar Mediana ou Média** para substituir `NaN` com base em campanhas semelhantes.
- **Manter `NaN`** e ajustar a análise para ignorar valores ausentes.

Essas abordagens ajudam a evitar conclusões distorcidas e garantem uma análise mais precisa.

In [14]:
# Colunas com valores de reproduções de vídeo que podem ser substituídos por 0, onde NaN significa 'sem reproduções'
colunas_reproducoes = [
    "Reproduções de 25% do vídeo",
    "Reproduções de 50% do vídeo",
    "Reproduções de 75% do vídeo",
    "Reproduções de 95% do vídeo",
    "Reproduções de 100% do vídeo",
]

# Substituição de valores NaN por zero
dados_anuncios[colunas_reproducoes] = dados_anuncios[colunas_reproducoes].fillna(0)

In [15]:
dados_anuncios.to_csv("miro_dados_limpos_desde_janeiro.csv", index=False) 